In [9]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# coding=utf-8

# Ensayos pulsos BL-2

Trato de reproducir las mediciones que se anlizan en `anl150807`. Los anchos temporales de los pulsos fueron:

pulser= np.array([50e-9, 75e-9, 100E-9, 125E-9, 150E-9, 175E-9, 200E-9, 225E-9, 250e-9, 275e-9,
                  3.0567943706e-07, 3.25173964034e-07, 3.47919077404e-07, 3.73028928851e-07, 3.98022673964e-07, 4.23560985145e-07 ])

## Parámetros del generador de pulsos BL-2



- Pulse out: usa salída negativa
- reference: INT
- frequency= 100 Hz

Los ajustes de tiempos apuntan a minimizar un notorio fenómeno de Gibbs:

- delay= 10 ms (single pulse)
- riseTime= 10 ns
- fallTime= 5 ns

La corriente a la entrada de la electrónica de luís debe ser de $1\mathrm{\mu A}$ pico-pico. Como uso una resistencia de $100\mathrm{k\Omega}$ ($1\times10^5\mathrm{\Omega}$), los pulsos deben ser de $100\mathrm{m V}$ ($1\times10^{-1}\mathrm{V}$) pico-pico.

- amplitude: ajusto perilla hasta 0,215 para lograr $100\mathrm{m V}$ p-p

`test.get_channel_scale('CH1'), test.get_channel_position('CH1'), test.get_channel_scale('CH2'), test.get_channel_position('CH2'), test.horizontal_main_scale(), test.horizontal_main_position(), test.trigger_level()`

(0.01, 7.48, 0.2, -2.0, '1.0E-7', '1.76E-7', -0.0648)

# Configuración TDS

In [10]:
# import tek3 as tek3
import tek3
tds2002b = tek3.TekScope() # creo una instancia de tek3.TekScope
import time

# refresca de lo importado cada vez que se hace ejecución
%load_ext autoreload
%autoreload 2

TEKTRONIX,TDS 2002B,C101891,CF:91.1CT FV:v22.11
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Inicialización de captura ante trigger
tds2002b.write('INITIATE_CLEAR')
tds2002b.set_data_encoding('RIBinary')
tds2002b.write('ACQuire:MODe SAMple') # test.write('ACQuire:MODe PEAKdetect') lo descarto pues manual dice que puede verse mas ruidoso  
tds2002b.write('ACQUIRE:STOPAFTER SEQUENCE')
tds2002b.write('ACQuire:STATE RUN')

In [19]:
# Valores actuales ch1, ch2, trigger
tds2002b.get_channel_scale('CH1'), tds2002b.get_channel_position('CH1'), tds2002b.get_channel_scale('CH2'), tds2002b.get_channel_position('CH2'), tds2002b.horizontal_main_scale(), tds2002b.horizontal_main_position(), tds2002b.trigger_level()

(0.01, 7.48, 0.5, -2.52, '1.0E-7', '1.76E-7', -0.0696)

In [5]:
# Configuración ch1, ch2, trigger
tds2002b.write('CH1:SCAle 0.01')
tds2002b.write('CH1:POSition 7.48')
tds2002b.write('CH2:SCAle 0.5')
tds2002b.write('CH2:POSition -2.52')
tds2002b.horizontal_main_scale(1.0E-7)
tds2002b.horizontal_main_position(1.76E-7)
trgLevel= -0.0696
tds2002b.trigger_source('CH1') # canal donde monitoreo pulso BL-2
tds2002b.trigger_slope('FALL')
tds2002b.trigger_level(trgLevel)

# Copio de PulTek

In [13]:
inst= tds2002b # declaro nombre intermendio del instrumento, espero colgar por dependencia de clase algún día.

def escalas(inst):
    # obtener una vez los límites verticales del canal
    pos1 = inst.get_channel_position(1)
    scale1 = inst.get_channel_scale(1)
    pos2 = inst.get_channel_position(2)
    scale2 = inst.get_channel_scale(2)
    return pos1, scale1, pos2, scale2


def param(inst):
    '''
    Grabo parámetros de la medición
    '''
    inst.load_setup() # Obtiene parámetros del osciloscopio
    sets= inst.dico
    [pos1, scale1, pos2, scale2]= escalas(inst)
    sets['POSITION1']= pos1
    sets['SCALE1']= scale1
    sets['POSITION2']= pos2
    sets['SCALE2']= scale2
    sets['ELECTRONICA']= 'Luis'
    return sets


def lectura(inst, adquisiciones, tiempoInicial):
    # la lectura en sí
    ch1= np.empty([adquisiciones, inst.dataCount ], dtype ='int8')
    ch2= np.empty([adquisiciones, inst.dataCount ], dtype ='int8')
    zeit= inst.Xaxis()
    for medicion in range(adquisiciones):
        print('\rmedición '+ str(medicion)+ ' de '+ str(adquisiciones), end=' ')
        inst.set_data_source(1) # canal 2
        ch1[medicion]= inst.singleAcq()
        inst.set_data_source(2) # canal 2
        ch2[medicion]= inst.singleAcq()
    print('\rmedición '+ str(medicion+ 1)+ ' de '+ str(adquisiciones))
    print('Tiempo transcurrido= {:.1f} min'.format((time.time()- tiempoInicial)/60) ) # tiempo transcurrido en segundos
    return zeit, ch1, ch2


def flName(camara, tipo, N):
    filePath= './data/'
    fileDate= time.strftime('%y%m%d')
    fileCamara= camara # 'n' nuestra = Campbelliana
    fileN= N
    fileExt= '.npy'
    fileType= tipo
    fileName= filePath+ fileType+ fileDate + fileCamara+ ('{:02n}'.format(fileN))+ fileExt
    return fileName


def adquiere(inst, medicionDia, settings, adquisiciones, descripcionDetector, descripcionArchivo):
    '''
    registra ambos canales en formato de multiples arrays
    '''
    tiempoInicial= time.time()
    # ch2 =lectura(adquisiciones, puntosAdquisicion, dataOffset, tiempoInicial)
    [zeit, ch1, ch2]= lectura(inst, adquisiciones, tiempoInicial)
    # PULSER INVIERTO SIGNO CH1: Mantengo el ánalisis imperturbado
    ch1*= -1
    # graba    
    np.savez_compressed(flName(descripcionDetector, descripcionArchivo, medicionDia), settings=settings, zeit=zeit, ch1=ch1, ch2=ch2)

In [23]:
inst= tds2002b
adquisiciones= 1280 # número de lecturas
# adquisiciones= 10 # número de lecturas
descripcionDetector= 'z'
descripcionArchivo= 'pul'

# settings= auxTDS2002B.param()
settings= param(inst)

for medicionDia in np.arange(7,8): # medición 9 a 11 (dos mediciones)
    adquiere(inst, medicionDia, settings, adquisiciones, descripcionDetector, descripcionArchivo)

medición 1280 de 1280
Tiempo transcurrido= 24.2 min


## Journal

Realicé mediciones `pul151005zxx.npy.npz` desde `xx=01` con un ancho de $50\mathrm{ns}$ en incrementos de $25\mathrm{ns}$ por adquisición.

Realicé mediciones `pul151006zxx.npy.npz` desde `xx=01` con un ancho de $175\mathrm{ns}$ en incrementos de $25\mathrm{ns}$ por adquisición.

Realicé mediciones `pul151007zxx.npy.npz`

`xx=01` ancho de $62.5\mathrm{ns}$

`xx=02` ancho de $87.5\mathrm{ns}$ ?

`xx=03` ancho de $87.5\mathrm{ns}$

`xx=04` ancho de $112.5\mathrm{ns}$

`xx=05` ancho de $137.5\mathrm{ns}$ ?


Realicé mediciones `pul151008zxx.npy.npz`

`xx=01` ancho de $137.5\mathrm{ns}$

`xx=02` ancho de $137.5\mathrm{ns}$ (en otra escala para evitar corte por sobretiro)

`xx=03` ancho de $162.5\mathrm{ns}$


Ahora con la alimentacion del estabilizador de tensión.

`xx=04` ancho de $37.5\mathrm{ns}$

`xx=05` ancho de $50\mathrm{ns}$

`xx=06` ancho de $62.5\mathrm{ns}$

`xx=07` ancho de $75\mathrm{ns}$

`xx=08` ancho de $87.5\mathrm{ns}$

`xx=09` ancho de $100\mathrm{ns}$

`xx=10` ancho de $112.5\mathrm{ns}$

`xx=11` ancho de $125\mathrm{ns}$

`xx=12` ancho de $137.5\mathrm{ns}$

`xx=13` ancho de $150\mathrm{ns}$

Continuo adquisiciones con estabilizada `pul151009zxx.npy.npz`

`xx=01` ancho de $162.5\mathrm{ns}$

`xx=02` ancho de $162.5\mathrm{ns}$ (bis, hasta que estabilizó el nivel del pulsador)

`xx=03` ancho de $175\mathrm{ns}$

`xx=04` ancho de $187.5\mathrm{ns}$

`xx=05` ancho de $200\mathrm{ns}$

`xx=06` ancho de $212.5\mathrm{ns}$

`xx=07` ancho de $225\mathrm{ns}$